In [1]:
import json

In [9]:
# JSON -> Python
# deserialization/decoding

s = '{"id": 100, "name": "John", "scores": [70, 80, 60]}'
x = json.loads(s)
x

{'id': 100, 'name': 'John', 'scores': [70, 80, 60]}

In [10]:
x

{'id': 100, 'name': 'John', 'scores': [70, 80, 60]}

In [11]:
s1 = json.dumps(x)
s1

'{"id": 100, "name": "John", "scores": [70, 80, 60]}'

In [12]:
json.dump(x, open('data.json', 'w'))

In [ ]:
# sftp, scp

In [13]:
y = json.load(open('data.json'))
y

{'id': 100, 'name': 'John', 'scores': [70, 80, 60]}

In [14]:
import pickle

In [15]:
import dill

In [ ]:
#json.load/dump
#json.dumps/loads


In [18]:
import requests

dburl = "https://dsci551-d5578-default-rtdb.firebaseio.com/"

# CRUD
# C - Create (PUT/put/post)
    # POST: generate random key first
    # PUT: overwrite
# R - Read (GET/get)
# U - Update (PATCH/patch)
# D - Delete (DELETE/delete)

resp = requests.get(dburl + '.json')
resp.text, resp.json(), json.loads(resp.text)



('{"temp":65,"today":"wed","weather":"cloudy"}',
 {'temp': 65, 'today': 'wed', 'weather': 'cloudy'},
 {'temp': 65, 'today': 'wed', 'weather': 'cloudy'})

In [19]:
# FYI
!curl -X GET 'https://dsci551-d5578-default-rtdb.firebaseio.com/.json'

{"temp":65,"today":"wed","weather":"cloudy"}

In [20]:
requests.put(dburl + 'users/100.json', '{"name": "John", "age": 20}')

<Response [200]>

In [21]:
requests.put(dburl + 'users/200.json', '{"name": "Bill"}')

<Response [200]>

In [22]:
requests.patch(dburl + 'users/200.json', '{"age": 30}')

<Response [200]>

In [23]:
requests.put(dburl + 'users/200/age.json', json.dumps(31))

<Response [200]>

In [24]:
requests.put(dburl + 'users/200/age.json', json=32)

<Response [200]>

In [26]:
resp = requests.patch(dburl + 'users/200/age.json', '33')
resp.text

'{\n  "error" : "Invalid data; couldn\'t parse JSON object. Are you sending a JSON object with valid key names?"\n}\n'

In [27]:
resp = requests.get(dburl + 'users/200.json')
resp.json()

{'age': 32, 'name': 'Bill'}

In [28]:
# add gender = "Male" and update age to 35, without changing name
requests.patch(dburl + 'users/200.json', json={'gender': 'Male', 'age': 35})
# upsert

<Response [200]>

In [29]:
requests.delete(dburl + 'users/200/gender.json')

<Response [200]>

In [31]:
requests.put(dburl + 'users/200.json', '{"name": "Bill", "age": 30}')

<Response [200]>

In [32]:
requests.put(dburl + 'users/300.json', '{"name": "David", "age": 40}')

<Response [200]>

In [33]:
requests.put(dburl + 'users/user400.json', '{"name": "Mary", "age": 28}')

<Response [200]>

In [34]:
requests.put(dburl + 'users/User500.json', '{"name": "Mary Smith", "age": 29}')

<Response [200]>

In [43]:
# find users whose id is 200 or later
# startAt: >= "200"
# endAt: <= "200"
# startAfter: >
# endBefore: <
# equalTo: =
# limitToFirst/Last = <number>
resp = requests.get(dburl + 'users.json?orderBy="$key"&startAt="200"&limitToLast=2')
resp.json()

# orderBy="$value"
# orderBy="name"
# orderBy="age"

# sqlalchemy, pymysql
# ORM (object relational mapping)

{'User500': {'age': 29, 'name': 'Mary Smith'},
 'user400': {'age': 28, 'name': 'Mary'}}

In [48]:
resp = requests.get(dburl + 'users.json?orderBy="name"&startAt="David"')
resp.json()

{'300': {'age': 40, 'name': 'David'},
 '100': {'age': 20, 'name': 'John'},
 'user400': {'age': 28, 'name': 'Mary'},
 'User500': {'age': 29, 'name': 'Mary Smith'}}

In [51]:
resp = requests.get(dburl + 'users/100/scores.json?orderBy="$value"&limitToFirst=1')
resp.json()


{'q3': 8}

In [56]:
class TypeValidator:
    """A descriptor to enforce type checking on an attribute."""
    def __init__(self, expected_type):
        self.expected_type = expected_type
        self.name = None

    def __set_name__(self, owner, name):
        """Called when the descriptor is assigned to a class attribute."""
        self.name = name

    def __get__(self, instance, owner):
        """Called when getting the attribute value."""
        if instance is None:
            return self
        # Get the actual value from the instance's dictionary
        return instance.__dict__.get(self.name, None)

    def __set__(self, instance, value):
        print('calling __set__', instance, value)
        """Called when setting the attribute value, with validation logic."""
        if not isinstance(value, self.expected_type):
            raise ValueError(f"Value for '{self.name}' must be of type {self.expected_type.__name__}")
        # Store the validated value in the instance's dictionary
        instance.__dict__[self.name] = value

# Class using the descriptor
class User:
    # Assigning descriptors as class attributes
    name = TypeValidator(str)
    age = TypeValidator(int)

    def __init__(self, name, age):
        self.name = name # The __set__ method is called here
        self.age = age   # The __set__ method is called here

# Usage example:
try:
    user = User("Alice", 30)
    print(f"{user.name} is {user.age} years old.") # The __get__ method is called here

    user.age = 31 # Valid
    print(f"Alice is now {user.age} years old.")

    user.age = "thirty-two" # Invalid, raises ValueError
except ValueError as e:
    print(f"Error: {e}")


calling __set__ <__main__.User object at 0x7956d103f4d0> Alice
calling __set__ <__main__.User object at 0x7956d103f4d0> 30
Alice is 30 years old.
calling __set__ <__main__.User object at 0x7956d103f4d0> 31
Alice is now 31 years old.
calling __set__ <__main__.User object at 0x7956d103f4d0> thirty-two
Error: Value for 'age' must be of type int


In [54]:
user.__dict__

{'name': 'Alice', 'age': 31}

In [55]:
User.__dict__

mappingproxy({'__module__': '__main__',
              'name': <__main__.TypeValidator at 0x7956d36a66f0>,
              'age': <__main__.TypeValidator at 0x7956d101fce0>,
              '__init__': <function __main__.User.__init__(self, name, age)>,
              '__dict__': <attribute '__dict__' of 'User' objects>,
              '__weakref__': <attribute '__weakref__' of 'User' objects>,
              '__doc__': None})

In [59]:
class C:
    x = 5

    def __init__(self, x):
        print(self.x)
        self.x = x # C.x is shadowed
        print(self.x)

c = C(10)
c.x, C.x

5
10


(10, 5)